In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
plt.style.use("nature")
from shapely import Point, LineString, Polygon
import tensorflow as tf
from waymo_open_dataset.protos import scenario_pb2

from waymo_devkit.query import get_intersection_stopSigns, get_intersection_circle, get_intersection_lanes
from waymo_devkit.query import get_egoTrajectory, get_vehicleTrajectories, get_scenario_metadata
from waymo_devkit.shape import construct_intersection_polygon
from waymo_devkit.identify import identify_conflict, identify_complex_conflicts
from waymo_devkit.visualize import visualize_gif, visualize_map, visualize_traj
from waymo_devkit.conflict import Conflict

In [3]:
ls

PET_minTTC_relationship_distributions.ipynb*
TA_distribution.ipynb*
csv/
find_conflicts.py*
find_scenarios.py*
follow_conflict_speed_distribution.ipynb*
maxReqDec_boxplot.ipynb*
minTTC_distribution.ipynb*
notebooks/
outputs/
pet_distribution.ipynb*
pkl/
speedProfile_confidenceInterval.ipynb*
test.ipynb*
training_20s/
visualize_traj.ipynb*
waymo_devkit/


In [4]:
with open('./pkl/cross_conflict_pet15s.pkl', 'rb') as file:
    all_cross_conflicts = pickle.load(file)

In [21]:
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# from shapely import Point, LineString, Polygon
# from waymo_devkit.query import get_intersection_stopSigns, get_intersection_circle, get_intersection_lanes

# INTERSECTION_CENTER_SIZE = 50
# INTERSECTION_CENTER_COLOR = "red"
# INTERSECTION_CENTER_MARKER = "*"
# INTERSECTION_RADIUS = 20

# IDENTIFIED_STOP_SIGN_COLOR = "orange"
# STOP_SIGN_COLOR = "red"
# STOP_SIGN_SIZE = 50
# STOP_SIGN_MARKER = "."

# for merge_index, row in tqdm(merge_PET_df.iterrows()):
#     # print(merge_index, row.tfrecord_index, row.scenario_index)  

#     tfrecord_path = f"./training_20s/training_20s.tfrecord-{row.tfrecord_index}-of-01000"
#     # get scenario object
#     dataset = tf.data.TFRecordDataset(tfrecord_path)
#     for index, scenario_proto in enumerate(dataset):
#         if index == int(row.scenario_index): 
#             scenario_proto = scenario_proto.numpy()
#             scenario = scenario_pb2.Scenario()
#             scenario.ParseFromString(scenario_proto)
#             break
    
#     fig, ax = plt.subplots(figsize=(5,5))
    
#     for mf in scenario.map_features:
#         if mf.WhichOneof('feature_data') == 'road_edge':
#             x, y = zip(*[(p.x, p.y) for p in mf.road_edge.polyline])
#             ax.plot(x, y, "black")
#         elif mf.WhichOneof('feature_data') == 'road_line':
#             x, y = zip(*[(p.x, p.y) for p in mf.road_line.polyline])
#             ax.plot(x, y, 'gray')
#         # elif mf.WhichOneof('feature_data') == 'lane':
#         #     x, y = zip(*[(p.x, p.y) for p in mf.lane.polyline])
#         #     plt.plot(x, y)        
#         elif mf.WhichOneof('feature_data') == 'stop_sign':
#             x, y = mf.stop_sign.position.x, mf.stop_sign.position.y
#             ax.scatter(x, y, c=STOP_SIGN_COLOR, s=STOP_SIGN_SIZE)
            
#     # draw the center of intersection
#     stopSigns = get_intersection_stopSigns(scenario, distance_threshold)
#     for ss in stopSigns:
#         ax.scatter(ss[1][0], ss[1][1], s=STOP_SIGN_SIZE, c=IDENTIFIED_STOP_SIGN_COLOR)
        
#     intersection_centerCoordinate, intersection_radius = get_intersection_circle(stopSigns, aggregation="max", buffer=2)
#     # display the center of the intersection
#     ax.scatter(intersection_centerCoordinate[0], intersection_centerCoordinate[1], 
#                 s=INTERSECTION_CENTER_SIZE, c=INTERSECTION_CENTER_COLOR, marker=INTERSECTION_CENTER_MARKER)
    
#     # draw the circular studying area 
#     theta = np.linspace(0, 2 * np.pi, 100) 
#     intersection_circle_x = intersection_centerCoordinate[0] + intersection_radius * np.cos(theta)  
#     intersection_circle_y = intersection_centerCoordinate[1] + intersection_radius * np.sin(theta) 
#     ax.plot(intersection_circle_x, intersection_circle_y, INTERSECTION_CENTER_COLOR)
    
#     intersection_area_coords = np.vstack([intersection_circle_x, intersection_circle_y]).T
#     assert intersection_area_coords.shape[1] == 2
#     intersection_area = Polygon(intersection_area_coords)
    
#     # draw the lane centers
#     inbound_lanes, outbound_lanes = get_intersection_lanes(scenario, intersection_area)
#     for lane in inbound_lanes:
#         ax.plot(lane[2][:,0], lane[2][:,1], "blue")  
#     for lane in outbound_lanes:
#         ax.plot(lane[2][:,0], lane[2][:,1], "green")  
    
#     c = all_merge_conflicts[merge_index] 
#     len_t = len(scenario.timestamps_seconds)
#     for track_index, track in enumerate(scenario.tracks):
#         if track.object_type == 1:
#             traj = []
#             # ego
#             if track_index == c.leader_index:
#                 for t in range(len_t):
#                     state = track.states[t]
#                     traj.append((state.center_x, state.center_y))
#                 traj = np.array(traj)
#                 ax.scatter(traj[:,0], traj[:,1], c='red', marker="s", s=1)
#             # agents
#             elif track_index == c.follower_index:
#                 for t in range(len_t):
#                     state = track.states[t]
#                     if state.valid:
#                         traj.append((state.center_x, state.center_y))
#                 traj = np.array(traj)
#                 ax.scatter(traj[:,0], traj[:,1], c='yellow', marker="s", s=1)
    
#     ax.set_xlim([intersection_centerCoordinate[0] - 50, intersection_centerCoordinate[0] + 50])
#     ax.set_ylim([intersection_centerCoordinate[1] - 50, intersection_centerCoordinate[1] + 50])
#     plt.savefig(f'./outputs/trajectories/merge_{row.tfrecord_index}-{int(row.scenario_index)}.png')
#     plt.close(fig) 

306it [01:27,  3.50it/s]


In [22]:
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# from shapely import Point, LineString, Polygon
# from waymo_devkit.query import get_intersection_stopSigns, get_intersection_circle, get_intersection_lanes

# INTERSECTION_CENTER_SIZE = 50
# INTERSECTION_CENTER_COLOR = "red"
# INTERSECTION_CENTER_MARKER = "*"
# INTERSECTION_RADIUS = 20

# IDENTIFIED_STOP_SIGN_COLOR = "orange"
# STOP_SIGN_COLOR = "red"
# STOP_SIGN_SIZE = 50
# STOP_SIGN_MARKER = "."

# for cross_index, row in tqdm(cross_PET_df.iterrows()):
#     # print(cross_index, row.tfrecord_index, row.scenario_index)  

#     tfrecord_path = f"./training_20s/training_20s.tfrecord-{row.tfrecord_index}-of-01000"
#     # get scenario object
#     dataset = tf.data.TFRecordDataset(tfrecord_path)
#     for index, scenario_proto in enumerate(dataset):
#         if index == int(row.scenario_index): 
#             scenario_proto = scenario_proto.numpy()
#             scenario = scenario_pb2.Scenario()
#             scenario.ParseFromString(scenario_proto)
#             break
    
#     fig, ax = plt.subplots(figsize=(5,5))
    
#     for mf in scenario.map_features:
#         if mf.WhichOneof('feature_data') == 'road_edge':
#             x, y = zip(*[(p.x, p.y) for p in mf.road_edge.polyline])
#             ax.plot(x, y, "black")
#         elif mf.WhichOneof('feature_data') == 'road_line':
#             x, y = zip(*[(p.x, p.y) for p in mf.road_line.polyline])
#             ax.plot(x, y, 'gray')
#         # elif mf.WhichOneof('feature_data') == 'lane':
#         #     x, y = zip(*[(p.x, p.y) for p in mf.lane.polyline])
#         #     plt.plot(x, y)        
#         elif mf.WhichOneof('feature_data') == 'stop_sign':
#             x, y = mf.stop_sign.position.x, mf.stop_sign.position.y
#             ax.scatter(x, y, c=STOP_SIGN_COLOR, s=STOP_SIGN_SIZE)
            
#     # draw the center of intersection
#     stopSigns = get_intersection_stopSigns(scenario, distance_threshold)
#     for ss in stopSigns:
#         ax.scatter(ss[1][0], ss[1][1], s=STOP_SIGN_SIZE, c=IDENTIFIED_STOP_SIGN_COLOR)
        
#     intersection_centerCoordinate, intersection_radius = get_intersection_circle(stopSigns, aggregation="max", buffer=2)
#     # display the center of the intersection
#     ax.scatter(intersection_centerCoordinate[0], intersection_centerCoordinate[1], 
#                 s=INTERSECTION_CENTER_SIZE, c=INTERSECTION_CENTER_COLOR, marker=INTERSECTION_CENTER_MARKER)
    
#     # draw the circular studying area 
#     theta = np.linspace(0, 2 * np.pi, 100) 
#     intersection_circle_x = intersection_centerCoordinate[0] + intersection_radius * np.cos(theta)  
#     intersection_circle_y = intersection_centerCoordinate[1] + intersection_radius * np.sin(theta) 
#     ax.plot(intersection_circle_x, intersection_circle_y, INTERSECTION_CENTER_COLOR)
    
#     intersection_area_coords = np.vstack([intersection_circle_x, intersection_circle_y]).T
#     assert intersection_area_coords.shape[1] == 2
#     intersection_area = Polygon(intersection_area_coords)
    
#     # draw the lane centers
#     inbound_lanes, outbound_lanes = get_intersection_lanes(scenario, intersection_area)
#     for lane in inbound_lanes:
#         ax.plot(lane[2][:,0], lane[2][:,1], "blue")  
#     for lane in outbound_lanes:
#         ax.plot(lane[2][:,0], lane[2][:,1], "green")  
    
#     c = all_cross_conflicts[cross_index] 
#     len_t = len(scenario.timestamps_seconds)
#     for track_index, track in enumerate(scenario.tracks):
#         if track.object_type == 1:
#             traj = []
#             # ego
#             if track_index == c.leader_index:
#                 for t in range(len_t):
#                     state = track.states[t]
#                     traj.append((state.center_x, state.center_y))
#                 traj = np.array(traj)
#                 ax.scatter(traj[:,0], traj[:,1], c='red', marker="s", s=1)
#             # agents
#             elif track_index == c.follower_index:
#                 for t in range(len_t):
#                     state = track.states[t]
#                     if state.valid:
#                         traj.append((state.center_x, state.center_y))
#                 traj = np.array(traj)
#                 ax.scatter(traj[:,0], traj[:,1], c='yellow', marker="s", s=1)
    
#     ax.set_xlim([intersection_centerCoordinate[0] - 50, intersection_centerCoordinate[0] + 50])
#     ax.set_ylim([intersection_centerCoordinate[1] - 50, intersection_centerCoordinate[1] + 50])
#     plt.savefig(f'./outputs/trajectories/cross_{row.tfrecord_index}-{int(row.scenario_index)}.png')
#     plt.close(fig) 

605it [02:52,  3.50it/s]


## Generate individual plot

In [ ]:
txt_path="./outputs/scenario_metadata/4stopSigns.csv"
metadatas = get_scenario_metadata(txt_path)
distance_threshold = 45
buffer = 10
pet = 10

for scene_id, (tfrecord_index, scenario_index) in enumerate(metadatas):
    # print(scene_id, (tfrecord_index, int(scenario_index)))
    scene_conflicts = []

    tfrecord_path = f"./training_20s/training_20s.tfrecord-{tfrecord_index}-of-01000"
    # get scenario object
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    for index, scenario_proto in enumerate(dataset):
        if index == int(scenario_index): 
            scenario_proto = scenario_proto.numpy()
            scenario = scenario_pb2.Scenario()
            scenario.ParseFromString(scenario_proto)
            break

    """ extract info in scenario """
    # get map elements
    stopSigns = get_intersection_stopSigns(scenario)
    intersection_centerCoordinate, intersection_radius = get_intersection_circle(stopSigns, buffer=buffer)
    intersection_polygon = construct_intersection_polygon(intersection_centerCoordinate, intersection_radius)

    inbound_lanes, outbound_lanes = get_intersection_lanes(scenario, intersection_polygon)

    # get trajectories
    egoTrajectory = get_egoTrajectory(scenario)
    vehicleTrajectories = get_vehicleTrajectories(scenario, intersection_polygon)
    # n_legs=3 if len(stopSigns)==3 else 4
    # visualize_map(scenario, tfrecord_index, int(scenario_index), n_legs, distance_threshold=distance_threshold, buffer=buffer)
    
    """ identify conflict """
    # AV-HV
    for veh_trajectory in vehicleTrajectories:
        conflict_type, c = identify_conflict(
            egoTrajectory, veh_trajectory,
            inbound_lanes, outbound_lanes, True, False, 
            intersection_circle=intersection_polygon, PET=15,
            center=intersection_centerCoordinate, radius=intersection_radius,
        )
        # @AV-HV: merge
        if conflict_type == "MERGE":
            scene_conflicts.append(c)
        # @AV-HV: cross
        elif conflict_type == "CROSS":
            scene_conflicts.append(c)
                        
    # HV-HV
    for i, veh_trajectory1 in enumerate(vehicleTrajectories):
        for j, veh_trajectory2 in enumerate(vehicleTrajectories):
            if i != j:
                conflict_type, c = identify_conflict(
                    veh_trajectory1, veh_trajectory2,
                    inbound_lanes, outbound_lanes, False, False, 
                    intersection_circle=intersection_polygon, PET=15,
                    center=intersection_centerCoordinate, radius=intersection_radius,
                )
                # @HV-HV: merge
                if conflict_type == "MERGE":
                    # print(tfrecord_index, int(scenario_index), conflict_type, egoTrajectory[0], veh_trajectory[0])
                    # NUM_MERGE += 1
                    scene_conflicts.append(c)
                # @HV-HV: cross
                elif conflict_type == "CROSS":
                    # print(tfrecord_index, int(scenario_index), conflict_type, egoTrajectory[0], veh_trajectory[0])
                    # NUM_CROSS += 1
                    scene_conflicts.append(c)

    # if exists complex conflict:
    if len(scene_conflicts) > 1:

    
        c = scene_conflicts[0]
        c = Conflict(
        leader_id=c['leader_id'],
        leader_index=c['leader_index'],
        leader_states=c['leader_states'],
        leader_time_at_conflict=c['leader_time_at_conflict'],
        follower_id=c['follower_id'],
        follower_index=c['follower_index'],
        follower_states=c['follower_states'],
        follower_time_at_conflict=c['follower_time_at_conflict'],
        PET=c['PET'],
        conflict_type=c['conflict_type'],
        leader_is_av=c['leader_is_av'],
        follower_is_av=c['follower_is_av'],
        center=c['center'], radius=c['radius']
        )
    
        fig, ax = plt.subplots(figsize=(4, 3))
    
        for mf in scenario.map_features:
            if mf.WhichOneof('feature_data') == 'road_edge':
                x, y = zip(*[(p.x, p.y) for p in mf.road_edge.polyline])
                ax.plot(x, y, "black", linewidth=1)
            elif mf.WhichOneof('feature_data') == 'road_line':
                x, y = zip(*[(p.x, p.y) for p in mf.road_line.polyline])
                ax.plot(x, y, 'gray')
            # elif mf.WhichOneof('feature_data') == 'lane':
            #     x, y = zip(*[(p.x, p.y) for p in mf.lane.polyline])
            #     plt.plot(x, y)        
            # elif mf.WhichOneof('feature_data') == 'stop_sign':
            #     x, y = mf.stop_sign.position.x, mf.stop_sign.position.y
            #     ax.scatter(x, y, c=STOP_SIGN_COLOR, s=STOP_SIGN_SIZE)
                
        # draw the center of intersection
        # stopSigns = get_intersection_stopSigns(scenario, distance_threshold)
        # for ss in stopSigns:
        #     ax.scatter(ss[1][0], ss[1][1], s=STOP_SIGN_SIZE, c=IDENTIFIED_STOP_SIGN_COLOR)
            
        intersection_centerCoordinate, intersection_radius = get_intersection_circle(stopSigns, aggregation="max", buffer=2)
        # display the center of the intersection
        # ax.scatter(intersection_centerCoordinate[0], intersection_centerCoordinate[1], 
        #             s=INTERSECTION_CENTER_SIZE, c=INTERSECTION_CENTER_COLOR, marker=INTERSECTION_CENTER_MARKER)
        
        # draw the circular studying area 
        # theta = np.linspace(0, 2 * np.pi, 100) 
        # intersection_circle_x = intersection_centerCoordinate[0] + intersection_radius * np.cos(theta)  
        # intersection_circle_y = intersection_centerCoordinate[1] + intersection_radius * np.sin(theta) 
        # ax.plot(intersection_circle_x, intersection_circle_y, INTERSECTION_CENTER_COLOR)
        
        # intersection_area_coords = np.vstack([intersection_circle_x, intersection_circle_y]).T
        # assert intersection_area_coords.shape[1] == 2
        # intersection_area = Polygon(intersection_area_coords)
        
        # draw the lane centers
        # inbound_lanes, outbound_lanes = get_intersection_lanes(scenario, intersection_area)
        # for lane in inbound_lanes:
        #     ax.plot(lane[2][:,0], lane[2][:,1], "blue")  
        # for lane in outbound_lanes:
        #     ax.plot(lane[2][:,0], lane[2][:,1], "green")  
        
        len_t = len(scenario.timestamps_seconds)
        for track_index, track in enumerate(scenario.tracks):
            if track.object_type == 1:
                traj = []
                # ego
                if track_index == c.leader_index:
                    for t in range(len_t):
                        state = track.states[t]
                        traj.append((state.center_x, state.center_y))
                    traj = np.array(traj)
                    ax.scatter(traj[:,0], traj[:,1], c=[range(len_t)], marker="s", s=0.5, vmax=200, vmin=0, cmap="viridis")
                elif track_index == c.follower_index:
                    ts = []
                    for t in range(len_t):
                        state = track.states[t]
                        if state.valid:
                            ts.append(t)
                            traj.append((state.center_x, state.center_y))
                    traj = np.array(traj)
                    ts = np.array(ts) * 0.1
                    g = ax.scatter(traj[:,0], traj[:,1], c=ts, marker="s", s=0.5, vmax=19.8, vmin=0, cmap="viridis")
        OFFSET = 50
        
        ax.set_xlim([intersection_centerCoordinate[0] - OFFSET, intersection_centerCoordinate[0] + OFFSET])
        ax.set_ylim([intersection_centerCoordinate[1] - OFFSET, intersection_centerCoordinate[1] + OFFSET])
        # ax.set_xlabel("X")
        # ax.set_ylabel("Y")
        plt.colorbar(g, label="Time (s)")
        plt.savefig(f"./outputs/trajectories/{tfrecord_index}-{scenario_index}.png", dpi=300)
        plt.close()
        # plt.show()
        # break

In [ ]:
txt_path="./outputs/scenario_metadata/4stopSigns.csv"
metadatas = get_scenario_metadata(txt_path)
distance_threshold = 45
buffer = 10
pet = 10

for scene_id, (tfrecord_index, scenario_index) in enumerate(metadatas):
    # print(scene_id, (tfrecord_index, int(scenario_index)))
    scene_conflicts = []

    tfrecord_path = f"./training_20s/training_20s.tfrecord-{tfrecord_index}-of-01000"
    # get scenario object
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    for index, scenario_proto in enumerate(dataset):
        if index == int(scenario_index): 
            scenario_proto = scenario_proto.numpy()
            scenario = scenario_pb2.Scenario()
            scenario.ParseFromString(scenario_proto)
            break

    """ extract info in scenario """
    # get map elements
    stopSigns = get_intersection_stopSigns(scenario)
    intersection_centerCoordinate, intersection_radius = get_intersection_circle(stopSigns, buffer=buffer)
    intersection_polygon = construct_intersection_polygon(intersection_centerCoordinate, intersection_radius)

    inbound_lanes, outbound_lanes = get_intersection_lanes(scenario, intersection_polygon)

    # get trajectories
    egoTrajectory = get_egoTrajectory(scenario)
    vehicleTrajectories = get_vehicleTrajectories(scenario, intersection_polygon)
    n_legs=3 if len(stopSigns)==3 else 4
    visualize_map(scenario, tfrecord_index, int(scenario_index), n_legs, distance_threshold=distance_threshold, buffer=buffer)